In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from tqdm.notebook import tqdm

In [2]:
raw_data = pd.read_csv("단지별 주소_200906.csv")

In [3]:
data = raw_data

data = data[["id", "주소"]]

In [4]:
data.head()

,id,주소
0,6993,충청북도 청주시 상당구 금천동 77-7
1,6997,충청북도 청주시 상당구 금천동 81
2,6998,충청북도 청주시 상당구 금천동 122-15
3,7001,충청북도 청주시 상당구 금천동 115-1
4,7004,충청북도 청주시 상당구 금천동 70-1


In [9]:
driver = webdriver.Chrome("chromedriver")

In [10]:
df = pd.DataFrame(columns=["id", "addr"]+[f"{yr}" for yr in range(2020, 1999, -1)])

In [18]:
for _, row in tqdm(data.iterrows()):
    dic = {}
    
    #주소를 parsing합니다.
    addr = row["주소"].split()
    gu = addr[2]
    dong = addr[3]
    lot1 = addr[-1].split("-")[0]
    if len(addr[-1].split("-")) == 2:
        lot2 = addr[-1].split("-")[1]
    else:
        lot2 = None
    if len(addr) == 6:
        ri = addr[-2]
    else:
        ri = None

    driver.get(("http://kras.chungbuk.go.kr/land_info/"
                "info/landprice/landprice.do"))
    
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//*[@id='sggnm']")))
        
    #시군구를 입력합니다.
    driver.find_element_by_xpath("//*[@id='sggnm']").send_keys(f"청주시{gu}")

    time.sleep(1)

    #동을 입력합니다.
    driver.find_element_by_xpath("//*[@id='umdnm']").send_keys(dong)

    time.sleep(1)

    #주소가 리까지라면
    if ri != None:
        #리를 입력합니다.
        driver.find_element_by_xpath("//*[@id='rinm']").clear()
        driver.find_element_by_xpath("//*[@id='rinm']").send_keys(ri)

    time.sleep(1)

    driver.find_element_by_xpath("//*[@id='textfield']").clear()
    driver.find_element_by_xpath("//*[@id='textfield2']").clear()
    #지번 주소를 입력합니다.
    if lot2 == None:
        driver.find_element_by_xpath("//*[@id='textfield']").send_keys(lot1)
    elif lot2 != None:
        driver.find_element_by_xpath("//*[@id='textfield']").send_keys(int(lot1))
        driver.find_element_by_xpath("//*[@id='textfield2']").send_keys(int(lot2))

    time.sleep(1)

    #주소가 완전히 입력되도록 "지번입력조회" 버튼을 클릭합니다.
    driver.execute_script("arguments[0].click();", driver.find_element_by_xpath("//*[@id='searchType3']"))

    time.sleep(1)

    #"검색" 버튼을 클릭합니다.
    driver.execute_script("arguments[0].click();", driver.find_element_by_xpath("//*[@id='searching']/a"))

    #표가 출력되기를 기다립니다.
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//*[@id='main-container']/div[2]/table/thead/tr/th[1]")))
        
    for i in range(2, 23):
        try:
            year = driver.find_element_by_xpath(f"//*[@id='main-container']/div[2]/table/tbody/tr[{i}]/td[1]").text
            value = driver.find_element_by_xpath(f"//*[@id='main-container']/div[2]/table/tbody/tr[{i}]/td[4]").text

            dic.update({year:value})
        except:
            break
    dic.update({"id":row["id"], "addr":row["주소"]})

    print(dic)

    df = df.append(dic, ignore_index=True)
    
    try:
        df.to_csv("공시지가.csv", encoding="euc-kr")
    except:
        continue

{'2020': '670,800 원', '2019': '652,000 원', '2018': '625,400 원', '2017': '574,400 원', '2016': '494,500 원', '2015': '487,700 원', '2014': '453,100 원', '2013': '430,000 원', '2012': '388,000 원', '2011': '388,000 원', '2010': '377,000 원', '2009': '367,000 원', '2008': '377,000 원', '2007': '380,000 원', '2006': '380,000 원', '2005': '370,000 원', '2004': '350,000 원', '2003': '320,000 원', '2002': '330,000 원', '2001': '360,000 원', '2000': '360,000 원', 'id': 6993, 'addr': '충청북도 청주시 상당구 금천동 77-7'}
{'2020': '645,000 원', '2019': '627,000 원', '2018': '601,300 원', '2017': '546,200 원', '2016': '470,200 원', '2015': '465,000 원', '2014': '432,000 원', '2013': '410,000 원', '2012': '370,000 원', '2011': '370,000 원', '2010': '360,000 원', '2009': '350,000 원', '2008': '276,000 원', '2007': '276,000 원', '2006': '276,000 원', '2005': '276,000 원', '2004': '252,000 원', '2003': '259,000 원', '2002': '261,000 원', '2001': '243,000 원', '2000': '243,000 원', 'id': 6997, 'addr': '충청북도 청주시 상당구 금천동 81'}



KeyboardInterrupt: 

In [19]:
df

,id,addr,2020,2019,2018,2017,2016,2015,2014,2013,...,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000
0,6993,충청북도 청주시 상당구 금천동 77-7,"670,800 원","652,000 원","625,400 원","574,400 원","494,500 원","487,700 원","453,100 원","430,000 원",...,"367,000 원","377,000 원","380,000 원","380,000 원","370,000 원","350,000 원","320,000 원","330,000 원","360,000 원","360,000 원"
1,6997,충청북도 청주시 상당구 금천동 81,"645,000 원","627,000 원","601,300 원","546,200 원","470,200 원","465,000 원","432,000 원","410,000 원",...,"350,000 원","276,000 원","276,000 원","276,000 원","276,000 원","252,000 원","259,000 원","261,000 원","243,000 원","243,000 원"
